In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
from queue import Queue
import json

import requests
from bs4 import BeautifulSoup

In [12]:
def scrape_article(slug, user_input_date):
    post = {'slug': str(slug)}
    url = "https://new.baguio.gov.ph/api/news/article"
    headers = {
        "authority": "new.baguio.gov.ph",
        "method": "POST",
        "path": "/api/news/article",
        "scheme": "https",
        "accept": "application/json, text/plain, */*",
        "accept-encoding": "gzip, deflate, br",
        "accept-language": "en-US,en;q=0.5",
        "authorization": "Bearer null",
        "content-length": "84",
        "content-type": "application/json",
        "origin": "https://new.baguio.gov.ph",
        "referer": f"https://new.baguio.gov.ph/news/{str(slug)}",
        "sec-ch-ua": '"Brave";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Windows"',
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "sec-gpc": "1",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
    }
    
    response = requests.post(url, json=post, headers=headers)
    
    title = ''
    date = ''
    content =''
    
    try:
        if response.status_code == 200:
            data = json.loads(response.text)
            for item in data["data"]:
                title = item['title']
                date = user_input_date
                content = item['body']
        else:
            print("Failed to fetch data")
    except NoSuchElementException:
        pass
    
    if len(title) == 0 or len(date) == 0  or len(content) == 0 :
        return ['empty', 'empty', 'empty']
    else:
        return [title, date, content]

In [18]:
def bn_navigate_scrape_website(user_input_date):
    user_input_date = str(user_input_date)
    headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}
    url = f'https://new.baguio.gov.ph/api/news/search?page=1&office=0&keyword=&from={user_input_date}&to={user_input_date}&admin=undefined&barangay=undefined&public=true'

    response = requests.get(url)
    links = []

    if response.status_code == 200:
        data = json.loads(response.text)
        for item in data["data"]:
            links.append(item["slug"])
    else:
        print("Failed to fetch data")

    if links:
        bn_news = {'title': [], 'date': [], 'content': []}

        for url in links:
            bn_news_list = scrape_article(url, user_input_date)
            bn_news['title'].append(bn_news_list[0])
            bn_news['date'].append(bn_news_list[1])
            bn_news['content'].append(bn_news_list[2])

        if bn_news:
            return bn_news
        else:
            return ['No data']

    else:
        return ['none']

In [19]:
test = bn_navigate_scrape_website('2023-04-24')

In [21]:
test

{'title': ['BAGUIO CITY STRESSES YOUTH PARTICIPATION IN STREETS FOR CHILDREN ROADSHOP',
  'BAGUIO’s SABSABONG TI MAYO LEVELS UP WITH FLORAL GASTRO FIESTA',
  'ACTING ON SUSTAINABLE TOURISM FOR THE ENVIRONMENT'],
 'date': ['2023-04-24', '2023-04-24', '2023-04-24'],
 'content': ['<p>The United Nations Development Programme (UNDP) and the City Government of Baguio through the City Planning, Development and Sustainability Office (CPDSO) and the City Engineering Office (CEO) conducted a Streets for Children Roadshop in Baguio City National High School - Hillside Annex aiming to promote livable communities from children’s perspective, April 19. </p><p>Sustainability and Development Officer Arch. Donna Tabangin said that one of the problem areas of the city is the streets and thus the roadshop aims to include the children and youth as codesigners of streets in the city. </p><p>“The roads, for example, are public spaces which are shared by everyone. And, these are used by kids and young people